###### Code - Import librairies

In [1]:
import glob, os    
from datetime import date

import warnings
warnings.filterwarnings('ignore')

# Plotly offline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Import lecture/stockage données
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import calmap
import folium
from IPython.core.display import display, HTML

###### Code - Palette couleurs

In [2]:
# Palette
colorGrey = '#393e46'
colorRed = '#ff2e63' 
colorCyan = '#21bf73' 
colorYellow = '#fe9801'

###### Code - Chargement des données

In [3]:
# On charge et map l'ensemble des csv en un seul
# dfGlobalDaily = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/*.csv"))))

# dfGlobalDailyLatest = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed/" + str(d1) + ".csv")
dfGlobalDailyLatest = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-18-2020.csv")

# Chargement des times series pour les cas confirmés
dfGlobalSeriesConfirmed = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")

# Chargement des times series pour les cas décédés
dfGlobalSeriesDeaths = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")

# Chargement des times series pour les cas soignés
dfGlobalSeriesRecovered = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")

## Comparatif du nombres de chaque cas au cours du temps

###### Code

In [12]:
# On récupère que le row FRANCE depuis les times series des cas confirmés
frSeriesConfirmed = dfGlobalSeriesConfirmed[(dfGlobalSeriesConfirmed['Country/Region'] == 'France') & (dfGlobalSeriesConfirmed['Province/State'] == 'France')]
frSeriesConfirmed = frSeriesConfirmed.drop(columns=['Lat', 'Long'])

# On récupère que le row FRANCE depuis les times series des cas décédés
frSeriesDeaths = dfGlobalSeriesDeaths[(dfGlobalSeriesConfirmed['Country/Region'] == 'France') & (dfGlobalSeriesDeaths['Province/State'] == 'France')]
frSeriesDeaths = frSeriesDeaths.drop(columns=['Lat', 'Long'])

# On récupère que le row FRANCE depuis les times series des cas soignés
frSeriesRecovered = dfGlobalSeriesRecovered[(dfGlobalSeriesConfirmed['Country/Region'] == 'France') & (dfGlobalSeriesRecovered['Province/State'] == 'France')]
frSeriesRecovered = frSeriesRecovered.drop(columns=['Lat', 'Long'])

# On rassemble l'ensemble de nos données précédents
frSeriesAll = pd.concat([frSeriesConfirmed, frSeriesDeaths, frSeriesRecovered])
frSeriesAll = frSeriesAll.drop(columns=['Province/State', 'Country/Region'])
frSeriesAll = frSeriesAll.reset_index()
frSeriesAll = frSeriesAll.drop(columns=['index'])

# On transpose notre matrice pour avoir une colonne date, une colone décédés, une colonne soignés, une colonne soignés
frSeriesAll = frSeriesAll.transpose()
frSeriesAll = frSeriesAll.rename(columns={0: 'Confirmed', 1: 'Deaths', 2: 'Recovered'})
frSeriesAll = frSeriesAll.reset_index()
frSeriesAll = frSeriesAll.rename(columns={'index' : 'Date'})

figFrSeriesAll = frSeriesAll.melt(id_vars="Date", 
                                 value_vars=['Confirmed', 'Deaths', 'Recovered'],
                                value_name='Nombre de cas',
                              var_name='type')

iplot(px.bar(figFrSeriesAll, 
             x="Date", 
             y="Nombre de cas", 
             color='type',
             title='France',
             color_discrete_sequence=[colorYellow, colorRed, colorCyan],
             barmode='group'))


## Taux de létalité et de guerison au cours du temps

In [13]:
# On rassemble par Date
temp = frSeriesAll.reset_index()
temp = temp.groupby(['Date'])['Confirmed', 'Deaths', 'Recovered'].sum().reset_index()

# Ajout colonne  taux mortalité
temp['Mortality Rate'] = round((temp['Deaths']/temp['Confirmed']), 3)*100
temp = temp.fillna({'Mortality Rate': 0.00})

# Ajout colonne taux de guerison
temp['Recovery Rate'] = round((temp['Recovered']/temp['Confirmed']), 3)*100
temp = temp.fillna({'Recovery Rate': 0.00})

# On converti notre string en Date
temp['Date'] = pd.to_datetime(temp['Date'], format='%m/%d/%y')

# On trie en fonction de la date
temp = temp.sort_values(by='Date', ascending=True)

# On transforme les données en 2 colonnes pour afficher à notre schéma
temp = temp.melt(id_vars='Date', value_vars=['Recovery Rate', 'Mortality Rate'], 
                 var_name='Ratio', value_name='Value')

iplot(px.line(temp, x="Date", y="Value", color='Ratio', log_y=True, 
              title='Taux de guerison & de létalité', color_discrete_sequence=[colorCyan, colorRed]))

source data : https://github.com/CSSEGISandData/COVID-19